In [ ]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 18)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test'

START: Concatenate and pickle Train, Validate and Test results

In [ ]:
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
run_type = 'train'
name_table = 'tbl_train'

# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
# run_type = 'validate'
# name_table = 'tbl_val'

# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
# run_type = 'test'
# name_table = 'tbl_test'

In [ ]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
              'grp(CAGR/UI)_mean':       ['mean', 'std',],
              'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
              'SPY_CAGR':                ['mean', 'std',],
              'SPY_CAGR/UI':             ['mean', 'std',],
              'SPY_CAGR/retnStd':        ['mean', 'std',],                           
              })
# tbl

In [ ]:
tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# tbl

In [ ]:
tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
tbl['pct_rank_d_CAGR/UI_m/s'] = tbl['dif_CAGR/UI_mean/std'].rank(pct=True)
tbl.head()
# len(tbl)

In [ ]:
tbl['run_type'] = run_type

In [ ]:
pickle_dump(tbl, path_data_dump, name_table)
# pickle_dump(tbl, path_data_dump, 'tbl.validate')
# pickle_dump(tbl, path_data_dump, 'tbl.test')

End: Concatenate and pickle Train, Validate and Test results

In [ ]:
# tbl_train = pickle_load(path_data_dump, 'tbl.train')
# # tbl_train.head()
# tbl_val = pickle_load(path_data_dump, 'tbl.validate')
# # tbl_val.head()
# tbl_test = pickle_load(path_data_dump, 'tbl.test')
# # tbl_test.head()


In [ ]:
# print(f'len(tbl_train): {len(tbl_train)}')
# print(f'len(tbl_val): {len(tbl_val)}')
# print(f'len(tbl_test): {len(tbl_test)}')

In [ ]:
tbl_concat = pd.concat([tbl_train, tbl_val, tbl_test])
tbl_concat.sort_values(by=["days_lookbacks", "days_eval", "sym_freq_cnt"])
# tbl_concat.head()

In [ ]:
tbl_concat.groupby(level=[0,1,2]).sum().sort_values('pct_rank_d_CAGR/UI_m/s', ascending=False)
tbl_concat.head()